In [1]:
import os
import shutil
import random
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms
# Path to your dataset (should contain one folder per class)
data_dir = r"/kaggle/input/anime-character-preprocess-dataset/dataset_balanced/train"   

transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}
dataset = datasets.ImageFolder(data_dir)

train_size = int(0.8 * len(dataset))
val_size = int(0.10 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_dataset.dataset = datasets.ImageFolder(root=data_dir, transform=transform["train"])
val_dataset.dataset   = datasets.ImageFolder(root=data_dir, transform=transform["val"])
test_dataset.dataset = datasets.ImageFolder(root=data_dir, transform=transform["val"])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

def train_model(model, criterion, optimizer, train_loader, val_loader, test_loader, num_epochs=15):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    
    print(" Starting training...")
    for epoch in range(num_epochs):
        print(f"\n Epoch {epoch + 1}/{num_epochs}")
        
        # Training
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
    
        avg_train_loss = running_loss / len(train_loader)
        train_acc = correct_train / total_train
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_acc)
    
        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
    
        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct / total
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)
    
        print(f" Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
        print(f" Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
    
    # 5. Final test evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    test_acc = correct / total * 100
    print(f"\n Final Test Accuracy: {test_acc:.2f}%")




In [3]:
num_classes = 173

model_resnet50 = models.resnet50(pretrained=True)

for param in model_resnet50.parameters():
    param.requires_grad = False
    
model_resnet50.fc = nn.Linear(model_resnet50.fc.in_features, num_classes)
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet50.fc.parameters(), lr=1e-3)
train_model(model = model_resnet50, criterion = criterion, optimizer = optimizer, train_loader = train_loader, val_loader = val_loader, test_loader = test_loader, num_epochs=15)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 187MB/s] 


 Starting training...

 Epoch 1/15
 Train Loss: 3.9413, Train Acc: 26.06%
 Val Loss: 2.6963, Val Acc: 50.03%

 Epoch 2/15
 Train Loss: 2.1440, Train Acc: 61.26%
 Val Loss: 1.8428, Val Acc: 63.91%

 Epoch 3/15
 Train Loss: 1.5129, Train Acc: 70.48%
 Val Loss: 1.5087, Val Acc: 66.83%

 Epoch 4/15
 Train Loss: 1.1899, Train Acc: 76.35%
 Val Loss: 1.3029, Val Acc: 71.35%

 Epoch 5/15
 Train Loss: 0.9992, Train Acc: 79.53%
 Val Loss: 1.2131, Val Acc: 72.07%

 Epoch 6/15
 Train Loss: 0.8684, Train Acc: 82.23%
 Val Loss: 1.1077, Val Acc: 74.55%

 Epoch 7/15
 Train Loss: 0.7678, Train Acc: 83.82%
 Val Loss: 1.0420, Val Acc: 74.71%

 Epoch 8/15
 Train Loss: 0.6808, Train Acc: 85.99%
 Val Loss: 1.0032, Val Acc: 74.10%

 Epoch 9/15
 Train Loss: 0.6175, Train Acc: 87.36%
 Val Loss: 0.9743, Val Acc: 75.48%

 Epoch 10/15
 Train Loss: 0.5615, Train Acc: 88.45%
 Val Loss: 0.9624, Val Acc: 75.70%

 Epoch 11/15
 Train Loss: 0.5172, Train Acc: 89.37%
 Val Loss: 0.9303, Val Acc: 75.92%

 Epoch 12/15
 Trai

In [4]:
torch.save(model_resnet50.state_dict(), "resnet50_finetuned.pth")
print(f" Model Resnet50 saved")


 Model Resnet50 saved


In [5]:
model_efficientnet_b0 = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Freeze toàn bộ layer (nếu chỉ muốn train classifier)
for param in model_efficientnet_b0.parameters():
    param.requires_grad = False

# Thay classifier cuối bằng số lớp của bạn
model_efficientnet_b0.classifier[1] = nn.Linear(model_efficientnet_b0.classifier[1].in_features, num_classes)


# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_efficientnet_b0.classifier[1].parameters(), lr=1e-3)
train_model(model = model_efficientnet_b0, criterion = criterion, optimizer = optimizer, train_loader = train_loader, val_loader = val_loader, test_loader = test_loader, num_epochs=15)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 144MB/s]


 Starting training...

 Epoch 1/15
 Train Loss: 4.0248, Train Acc: 31.28%
 Val Loss: 2.9646, Val Acc: 56.36%

 Epoch 2/15
 Train Loss: 2.4131, Train Acc: 64.92%
 Val Loss: 2.0604, Val Acc: 66.17%

 Epoch 3/15
 Train Loss: 1.7146, Train Acc: 73.62%
 Val Loss: 1.6807, Val Acc: 69.59%

 Epoch 4/15
 Train Loss: 1.3574, Train Acc: 77.97%
 Val Loss: 1.4594, Val Acc: 71.57%

 Epoch 5/15
 Train Loss: 1.1504, Train Acc: 80.49%
 Val Loss: 1.3312, Val Acc: 72.07%

 Epoch 6/15
 Train Loss: 0.9783, Train Acc: 83.04%
 Val Loss: 1.2409, Val Acc: 73.00%

 Epoch 7/15
 Train Loss: 0.8717, Train Acc: 84.93%
 Val Loss: 1.1641, Val Acc: 74.88%

 Epoch 8/15
 Train Loss: 0.7771, Train Acc: 86.60%
 Val Loss: 1.1138, Val Acc: 74.93%

 Epoch 9/15
 Train Loss: 0.6978, Train Acc: 87.88%
 Val Loss: 1.0759, Val Acc: 75.43%

 Epoch 10/15
 Train Loss: 0.6348, Train Acc: 88.83%
 Val Loss: 1.0440, Val Acc: 75.70%

 Epoch 11/15
 Train Loss: 0.5845, Train Acc: 89.93%
 Val Loss: 1.0185, Val Acc: 75.37%

 Epoch 12/15
 Trai

In [6]:
torch.save(model_efficientnet_b0.state_dict(), "EfficientnetB0_finetuned.pth")
print(f" Model EfficientnetB0_finetuned saved")


 Model EfficientnetB0_finetuned saved


In [7]:
from torchvision import transforms
data_dir = r"/kaggle/input/anime-character-preprocess-dataset/dataset_balanced/train"   

vit_weights = models.ViT_B_16_Weights.IMAGENET1K_V1
transform_vit = vit_weights.transforms()

dataset = datasets.ImageFolder(data_dir)

train_size = int(0.8 * len(dataset))
val_size = int(0.10 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset_vit, val_dataset_vit, test_dataset_vit = random_split(dataset, [train_size, val_size, test_size])
train_dataset_vit.dataset = datasets.ImageFolder(root=data_dir, transform=transform_vit)
val_dataset_vit.dataset   = datasets.ImageFolder(root=data_dir, transform=transform_vit)
test_dataset_vit.dataset = datasets.ImageFolder(root=data_dir, transform=transform_vit)

train_loader_vit = DataLoader(train_dataset_vit, batch_size=64, shuffle=True, num_workers=2)
val_loader_vit   = DataLoader(val_dataset_vit, batch_size=64, shuffle=False, num_workers=2)
test_loader_Vit  = DataLoader(test_dataset_vit, batch_size=64, shuffle=False, num_workers=2)

In [8]:
num_classes = 173

model_Vision_Transformer = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)

model_Vision_Transformer.heads.head = nn.Linear(model_Vision_Transformer.heads.head.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelmodel_Vision_Transformer= model_Vision_Transformer.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model_Vision_Transformer.heads.head.parameters(), lr=3e-4, weight_decay=0.01)
train_model(model = model_Vision_Transformer, criterion = criterion, optimizer = optimizer, train_loader = train_loader_vit, val_loader = val_loader_vit, test_loader = test_loader_Vit, num_epochs=15)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 210MB/s]  


 Starting training...

 Epoch 1/15
 Train Loss: 4.4777, Train Acc: 18.45%
 Val Loss: 3.8539, Val Acc: 37.63%

 Epoch 2/15
 Train Loss: 3.2737, Train Acc: 52.75%
 Val Loss: 2.9968, Val Acc: 56.31%

 Epoch 3/15
 Train Loss: 2.5025, Train Acc: 66.83%
 Val Loss: 2.4325, Val Acc: 63.64%

 Epoch 4/15
 Train Loss: 1.9949, Train Acc: 73.96%
 Val Loss: 2.0506, Val Acc: 69.48%

 Epoch 5/15
 Train Loss: 1.6437, Train Acc: 78.49%
 Val Loss: 1.7885, Val Acc: 72.89%

 Epoch 6/15
 Train Loss: 1.3924, Train Acc: 82.01%
 Val Loss: 1.5928, Val Acc: 75.10%

 Epoch 7/15
 Train Loss: 1.2030, Train Acc: 84.35%
 Val Loss: 1.4444, Val Acc: 77.25%

 Epoch 8/15
 Train Loss: 1.0548, Train Acc: 86.73%
 Val Loss: 1.3211, Val Acc: 78.90%

 Epoch 9/15
 Train Loss: 0.9352, Train Acc: 88.13%
 Val Loss: 1.2240, Val Acc: 80.50%

 Epoch 10/15
 Train Loss: 0.8371, Train Acc: 89.62%
 Val Loss: 1.1455, Val Acc: 81.05%

 Epoch 11/15
 Train Loss: 0.7532, Train Acc: 90.97%
 Val Loss: 1.0787, Val Acc: 81.05%

 Epoch 12/15
 Trai

In [9]:
# Sau khi train xong
torch.save(model_Vision_Transformer.state_dict(), "vit_model.pth")
print(" Saved ViT model weights to vit_model.pth")

 Saved ViT model weights to vit_model.pth
